# EdgeLLM API Testing Notebook

This notebook tests the EdgeLLM multi-tenant API gateway.

**Demo API Keys:**
- Admin: `edgellm-admin-demo-key-12345`
- User: `edgellm-user-demo-key-67890`

In [1]:
import requests
import json
from pprint import pprint

# Configuration
API_BASE_URL = "http://localhost:8000"
ADMIN_KEY = "edgellm-admin-demo-key-12345"
USER_KEY = "edgellm-user-demo-key-67890"

def api_request(method, endpoint, api_key=ADMIN_KEY, data=None):
    """Helper function for API requests."""
    headers = {
        "Authorization": f"Bearer {api_key}",
        "Content-Type": "application/json"
    }
    url = f"{API_BASE_URL}{endpoint}"
    
    if method == "GET":
        response = requests.get(url, headers=headers)
    elif method == "POST":
        response = requests.post(url, headers=headers, json=data)
    elif method == "DELETE":
        response = requests.delete(url, headers=headers)
    
    return response

print("EdgeLLM API Client Ready!")

EdgeLLM API Client Ready!


## 1. Check API Health

In [2]:
# Check API root endpoint
response = requests.get(f"{API_BASE_URL}/")
print("API Root:")
pprint(response.json())

API Root:
{'documentation': '/docs',
 'features': ['BitNet 1.58-bit quantization',
              '15.5x lower jitter',
              'Edge deployment ready'],
 'health': '/health',
 'service': 'EdgeLLM API Service',
 'status': 'running',
 'version': '1.0.0'}


In [3]:
# Check health endpoint
response = requests.get(f"{API_BASE_URL}/health")
print("Health Check:")
pprint(response.json())

Health Check:
{'edgellm_backend': 'unreachable',
 'status': 'healthy',
 'timestamp': '2026-01-12T17:13:05.811082'}


## 2. Test API Key Authentication

In [4]:
# Test with valid admin key
response = api_request("GET", "/api/stats", api_key=ADMIN_KEY)
print(f"Admin Key Status: {response.status_code}")
print("Stats:")
pprint(response.json())

Admin Key Status: 200
Stats:
{'average_response_time': 0.0,
 'last_24h_requests': 0,
 'requests_by_endpoint': {},
 'requests_by_model': {},
 'total_requests': 0}


In [5]:
# Test with valid user key
response = api_request("GET", "/api/stats", api_key=USER_KEY)
print(f"User Key Status: {response.status_code}")
print("Stats:")
pprint(response.json())

User Key Status: 200
Stats:
{'average_response_time': 0.0,
 'last_24h_requests': 0,
 'requests_by_endpoint': {},
 'requests_by_model': {},
 'total_requests': 0}


In [6]:
# Test with invalid key
response = api_request("GET", "/api/stats", api_key="invalid-key")
print(f"Invalid Key Status: {response.status_code}")
print("Response:", response.json())

Invalid Key Status: 401
Response: {'detail': 'Invalid API key'}


## 3. List API Keys (Admin Only)

In [7]:
# List all API keys (admin)
response = api_request("GET", "/api/keys", api_key=ADMIN_KEY)
print(f"List Keys Status: {response.status_code}")
print("\nAPI Keys:")
pprint(response.json())

List Keys Status: 200

API Keys:
{'api_keys': [{'created_at': '2026-01-12T16:11:33.832657',
               'id': 1,
               'is_active': True,
               'key_preview': 'edgellm-admin-demo-k...',
               'name': 'Demo Admin Key',
               'rate_limit': 1000,
               'role': 'admin'},
              {'created_at': '2026-01-12T16:11:33.836135',
               'id': 2,
               'is_active': True,
               'key_preview': 'edgellm-user-demo-ke...',
               'name': 'Demo User Key',
               'rate_limit': 100,
               'role': 'user'}],
 'total': 2}


In [8]:
# Try to list keys with user key (should fail)
response = api_request("GET", "/api/keys", api_key=USER_KEY)
print(f"User trying to list keys - Status: {response.status_code}")
print("Response:", response.json())

User trying to list keys - Status: 403
Response: {'detail': 'Admin access required'}


## 4. Create New API Key (Admin Only)

In [9]:
# Create a new API key
new_key_data = {
    "name": "Test Key from Notebook",
    "role": "user",
    "rate_limit": 50
}

response = api_request("POST", "/api/keys", api_key=ADMIN_KEY, data=new_key_data)
print(f"Create Key Status: {response.status_code}")

if response.status_code == 200:
    new_key = response.json()
    print("\nNew API Key Created:")
    pprint(new_key)
    TEST_KEY = new_key["api_key"]
    print(f"\nSaved as TEST_KEY: {TEST_KEY[:30]}...")
else:
    print("Response:", response.json())
    TEST_KEY = None

Create Key Status: 200

New API Key Created:
{'api_key': 'edgellm-v7Z-PKqx6ySSAOClcF7Wnn0YYNh8JwzSx9daF8xVOrM',
 'created_at': '2026-01-12T16:13:05.960657',
 'name': 'Test Key from Notebook',
 'rate_limit': 50,
 'role': 'user'}

Saved as TEST_KEY: edgellm-v7Z-PKqx6ySSAOClcF7Wnn...


## 5. Test New Key

In [10]:
# Test the newly created key
if TEST_KEY:
    response = api_request("GET", "/api/stats", api_key=TEST_KEY)
    print(f"New Key Test Status: {response.status_code}")
    print("Stats:")
    pprint(response.json())
else:
    print("No TEST_KEY available - create one first")

New Key Test Status: 200
Stats:
{'average_response_time': 0.0,
 'last_24h_requests': 0,
 'requests_by_endpoint': {},
 'requests_by_model': {},
 'total_requests': 0}


## 6. Get Detailed Stats

In [11]:
# Get detailed stats
response = api_request("GET", "/api/stats/detailed", api_key=ADMIN_KEY)
print(f"Detailed Stats Status: {response.status_code}")
print("\nDetailed Stats:")
pprint(response.json())

Detailed Stats Status: 200

Detailed Stats:
{'api_key_name': 'Demo Admin Key',
 'api_key_role': 'admin',
 'average_response_time': 0.0,
 'first_request': None,
 'last_request': None,
 'max_response_time': 0.0,
 'min_response_time': 0.0,
 'rate_limit': 1000,
 'rate_limit_usage_percent': 0.0,
 'requests_by_endpoint': {},
 'requests_by_model': {},
 'total_requests': 0,
 'total_requests_24h': 0,
 'total_requests_7d': 0}


## 7. Admin User Statistics

In [12]:
# Get all user statistics (admin only)
response = api_request("GET", "/api/admin/stats/users", api_key=ADMIN_KEY)
print(f"User Stats Status: {response.status_code}")
print("\nAll User Statistics:")
pprint(response.json())

User Stats Status: 200

All User Statistics:
{'summary': {'avg_response_time': 0.0,
             'total_requests': 0,
             'total_requests_24h': 0,
             'total_requests_7d': 0,
             'total_users': 3},
 'users': [{'avg_response_time': 0,
            'created_at': '2026-01-12T16:11:33.832657',
            'id': 1,
            'key_preview': 'edgellm-admin-demo-k...',
            'last_request': None,
            'name': 'Demo Admin Key',
            'rate_limit': 1000,
            'rate_limit_usage': 0.0,
            'requests_24h': 0,
            'requests_7d': 0,
            'requests_by_endpoint': {},
            'requests_by_model': {},
            'requests_this_hour': 0,
            'role': 'admin',
            'total_requests': 0},
           {'avg_response_time': 0,
            'created_at': '2026-01-12T16:11:33.836135',
            'id': 2,
            'key_preview': 'edgellm-user-demo-ke...',
            'last_request': None,
            'name': 'Demo Us

## 8. Test Chat Completion (Mock - EdgeLLM backend needed)

In [13]:
# Test chat completion endpoint
# Note: This requires the EdgeLLM inference server to be running

chat_data = {
    "model": "smollm-135m",
    "messages": [
        {"role": "system", "content": "You are a helpful assistant."},
        {"role": "user", "content": "Hello! What is 2+2?"}
    ],
    "temperature": 0.7
}

print("Testing chat completion...")
print("Note: This requires EdgeLLM inference server to be running")
print()

try:
    response = api_request("POST", "/api/chat", api_key=USER_KEY, data=chat_data)
    print(f"Chat Status: {response.status_code}")
    if response.status_code == 200:
        print("\nChat Response:")
        pprint(response.json())
    else:
        print("Error:", response.json())
except Exception as e:
    print(f"Error connecting to chat endpoint: {e}")
    print("Make sure the EdgeLLM inference server is running.")

Testing chat completion...
Note: This requires EdgeLLM inference server to be running

Chat Status: 500
Error: {'detail': 'Error in chat: All connection attempts failed'}


## 9. Rate Limiting Test

In [14]:
# Test rate limiting (make multiple requests)
print("Testing rate limiting (10 quick requests)...")
print()

results = []
for i in range(10):
    response = api_request("GET", "/api/stats", api_key=USER_KEY)
    results.append(response.status_code)
    
print(f"Request results: {results}")
print(f"Success: {results.count(200)}, Rate Limited: {results.count(429)}")

Testing rate limiting (10 quick requests)...

Request results: [200, 200, 200, 200, 200, 200, 200, 200, 200, 200]
Success: 10, Rate Limited: 0


## 10. Clean Up - Revoke Test Key

In [15]:
# List keys to find the test key
response = api_request("GET", "/api/keys", api_key=ADMIN_KEY)
keys = response.json().get("api_keys", [])

print("Current API Keys:")
for key in keys:
    print(f"  - {key['name']}: {key['key_preview']} (active: {key['is_active']})")

Current API Keys:
  - Demo Admin Key: edgellm-admin-demo-k... (active: True)
  - Demo User Key: edgellm-user-demo-ke... (active: True)
  - Test Key from Notebook: edgellm-v7Z-PKqx6ySS... (active: True)


In [16]:
# Revoke the test key (optional)
if TEST_KEY:
    # Get the key preview (first 20 chars)
    key_preview = TEST_KEY[:20]
    
    response = api_request("DELETE", f"/api/keys/{key_preview}", api_key=ADMIN_KEY)
    print(f"Revoke Key Status: {response.status_code}")
    print("Response:", response.json())
else:
    print("No TEST_KEY to revoke")

Revoke Key Status: 200
Response: {'message': 'API key revoked successfully'}


## Summary

This notebook demonstrates:

1. **Health Checks** - API status and health endpoints
2. **Authentication** - Bearer token authentication with API keys
3. **Authorization** - Admin vs User role separation
4. **API Key Management** - Create, list, and revoke keys
5. **Usage Statistics** - Basic and detailed stats
6. **Rate Limiting** - Request throttling per API key
7. **Chat Completion** - LLM inference (requires EdgeLLM server)

### API Endpoints Summary

| Endpoint | Method | Auth | Description |
|----------|--------|------|-------------|
| `/` | GET | No | API info |
| `/health` | GET | No | Health check |
| `/api/stats` | GET | Yes | Usage stats |
| `/api/stats/detailed` | GET | Yes | Detailed stats |
| `/api/keys` | GET | Admin | List all keys |
| `/api/keys` | POST | Admin | Create new key |
| `/api/keys/{preview}` | DELETE | Admin | Revoke key |
| `/api/chat` | POST | Yes | Chat completion |
| `/api/generate` | POST | Yes | Text generation |
| `/api/models` | GET | Yes | List models |